In [ ]:
#카카오맵 리뷰, 별점 크롤링


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

review_lists = []
star_widths = []

for i in detailpagelink_list:
    review_list = []
    star_width = []

    options = webdriver.ChromeOptions()

    prefs = {'download.default_directory': '/home/ted/dev_ws/EDA/eda-repo-5/1227',  ##수정
             'download.prompt_for_download': False}
    options.add_experimental_option('prefs', prefs)

    url = i
    driver = webdriver.Chrome(service=Service("/home/ted/dev_ws/EDA/driver/chromedriver-linux64/chromedriver"), ##수정
                               options=options)


    driver.get(url)
    time.sleep(1)

    try:
        link_more = driver.find_element(By.CLASS_NAME, "link_more")

        if link_more.text == "후기 더보기":
            while True:
                link_more = driver.find_element(By.CLASS_NAME, "link_more")
                if link_more.text == "후기 접기":
                    break

                link_more.click()
                time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"})
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"})
        stars = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"})

        for review in reviews:
            review_text = review.get_text(strip=True)
            review_text = review_text.replace("더보기", "")
            review_list.append(review_text)
        
        for star in stars:
            style = star.get("style", "")
            if "width" in style:
                width_percentage = int(style.split("width:")[1].split("%")[0].strip())
                star_rating = width_percentage / 20  # 100% -> 5점, 80% -> 4점 등
                star_width.append(star_rating)


    except Exception as e:
        print(f"Error processing URL {url}: {e}")

    finally:
        driver.quit()

#review_lists = [[review] for review in review_list]
    review_lists.append(review_list)
    star_widths.append(star_width)
